# Lighting Wildfires

#### Dependencies

In [1]:
from config import gkey
import gmaps
import numpy as np
import pandas as pd
import requests
from datetime import datetime as dt
from datetime import timedelta as td
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import csv

## Create DataFrame from csv 
Selecting perferred column to speed up compute time

In [2]:
# Raw Data
raw_data = pd.read_csv("./Wildfire Data/FW_Veg_Rem_Combined.csv")
print(raw_data.shape)

# Only selecting the columns that i want
df1 = raw_data[['fire_size', 'fire_size_class', 'stat_cause_descr', 'latitude', 'longitude', 'state', 'disc_date_final', 'cont_date_final' ]]
print(df1.shape)

# Filtering for 'lighting'
df2 = df1.loc[df1['stat_cause_descr'] == 'Lightning',:].copy()
print(df2.shape)

(55367, 43)
(55367, 8)
(8218, 8)


## Converting date columns to datetime
Creating putout_days columns by taking disc_date_final from cont_date_final

In [3]:
df2['disc_date_final'] = pd.to_datetime(df2['disc_date_final']).dt.date.copy()
df2['cont_date_final'] = pd.to_datetime(df2['cont_date_final']).dt.date.copy()
df2['putout_days'] = df2['cont_date_final'] - df2['disc_date_final']
df2.count()

fire_size           8218
fire_size_class     8218
stat_cause_descr    8218
latitude            8218
longitude           8218
state               8218
disc_date_final     6497
cont_date_final     6181
putout_days         6165
dtype: int64

## Dropping all row with NULL/NAT value in the datetime columns

In [4]:
df3 = df2
df3.dropna(inplace=True)
df3.count()

fire_size           6165
fire_size_class     6165
stat_cause_descr    6165
latitude            6165
longitude           6165
state               6165
disc_date_final     6165
cont_date_final     6165
putout_days         6165
dtype: int64

## Saving to CSV

In [5]:
df3.to_csv("./resources - kdmcc/assets/dataKd/lightingFireRaw.csv", index=False)

In [6]:
# Renaming columns
df4 = df3.rename(columns={
    "fire_size": "Fire Size" ,
    "fire_size_class": "Fire Size Class",
    "stat_cause_descr": "Cause Stat",
    "latitude": "Lat",
    "longitude": "Lng",
    "state": "State",
    "disc_date_final": "Reported Date",
    "cont_date_final": "Resolved Date",
    "putout_days": "Days to Resolve" 
})
df4.to_csv("./resources - kdmcc/data/lightingFire.csv", index=False)

In [7]:
import json

df4.to_json("./resources - kdmcc/data/lightingFire.json", orient='records')
data_json = df4.to_json(orient='records')
parsed = json.loads(data_json)
print(json.dumps(parsed, indent=4))

[
    {
        "Fire Size": 1420.0,
        "Fire Size Class": "F",
        "Cause Stat": "Lightning",
        "Lat": 33.2418,
        "Lng": -104.9122,
        "State": "NM",
        "Reported Date": 773107200000,
        "Resolved Date": 773193600000,
        "Days to Resolve": 86400000
    },
    {
        "Fire Size": 5.0,
        "Fire Size Class": "B",
        "Cause Stat": "Lightning",
        "Lat": 40.8347,
        "Lng": -112.8064,
        "State": "UT",
        "Reported Date": 1155513600000,
        "Resolved Date": 1155600000000,
        "Days to Resolve": 86400000
    },
    {
        "Fire Size": 1.5,
        "Fire Size Class": "B",
        "Cause Stat": "Lightning",
        "Lat": 39.205,
        "Lng": -105.1883333,
        "State": "CO",
        "Reported Date": 838771200000,
        "Resolved Date": 838857600000,
        "Days to Resolve": 86400000
    },
    {
        "Fire Size": 2.5,
        "Fire Size Class": "B",
        "Cause Stat": "Lightning",
        "Lat"

# Heatmap of Fire Size

In [8]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

- Convert Lat and Lng into a varable named locations cast datatype as float
- Convert Fire Size into a varable named fire_size cast datatype as float

In [9]:
# Store 'Lat' and 'Lng' into  locations 
locations = df4[["Lat", "Lng"]].astype(float)

# Convert Fire Size to float and store
fire_size = df4["Fire Size"].astype(float)

In [10]:
fire_size

36        1420.0
40           5.0
64           1.5
79           2.5
84         108.0
          ...   
55327     4472.0
55328    20621.0
55329    49100.0
55330     9211.0
55331     3082.0
Name: Fire Size, Length: 6165, dtype: float64

In [ ]:
# Create a poverty Heatmap layer
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights=fire_size, 
#                                  dissipating=False, max_intensity=10,
#                                  point_radius = 2)

# fig.add_layer(heat_layer)
# fig.add_layer(gmaps.heatmap_layer(locations))

# fig

In [11]:
import plotly.express as px


In [15]:
import pandas as px

us_cities = df4

import plotly.express as px

fig = px.scatter_mapbox(us_cities, lat="Lat", lon="Lng", hover_name="Cause Stat", hover_data=["State", "Fire Size"],
                        color_discrete_sequence=["blue"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
